In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_file = 'train_data.csv'
test_file = 'test_data.csv'

data_path = '/opt/ml/input/data/train_dataset'
train_path = os.path.join(data_path, train_file)
test_path = os.path.join(data_path, test_file)

In [ ]:
train_df = pd.read_csv(train_path, parse_dates=['Timestamp'])
test_df = pd.read_csv(test_path, parse_dates=['Timestamp'])

In [ ]:
train_df['tmp_index'] = train_df.index

In [ ]:
tmp_df = train_df[['userID', 'testId', 'Timestamp', 'tmp_index']].shift(1)

In [ ]:
tmp_df['tmp_index'] += 1

In [ ]:
train_df = train_df.merge(tmp_df, how='left', on=['userID', 'testId', 'tmp_index'])

In [ ]:
train_df['prior_elapsed'] = (train_df.Timestamp_x - train_df.Timestamp_y).dt.seconds

In [ ]:
train_df['prior_elapsed'].plot(kind='kde')


In [ ]:
train_df['log_prior'] = np.log1p(train_df['prior_elapsed'])

In [ ]:
train_df['log_prior'].plot(kind='kde')

In [ ]:
Q1 = train_df['prior_elapsed'].quantile(0.25)
Q3 = train_df['prior_elapsed'].quantile(0.75)

In [ ]:
Q1

In [ ]:
Q3

In [ ]:
len(train_df)

In [ ]:
(train_df['prior_elapsed'] > 65).sum()

In [ ]:
470314/ 2266586

In [ ]:
IQR = Q3 - Q1

In [ ]:
IQR

In [ ]:
low_bound = Q1 - 1.5 * IQR 
upper_bound = Q3 + 1.5 * IQR

In [ ]:
low_bound

In [ ]:
upper_bound

In [ ]:
(train_df['prior_elapsed'] > upper_bound).sum()

In [ ]:
193417 / 2266586

In [ ]:
Q99 = train_df['prior_elapsed'].quantile(0.98)

In [ ]:
Q99

In [ ]:
train_df['prior_elapsed'].median()

In [ ]:
train_df[train_df['prior_elapsed'] <= 260]['prior_elapsed'].median()

In [ ]:
train_df.groupby('assessmentItemID')['prior_elapsed'].get_group('A010001001')

In [ ]:
train_df.loc[train_df['assessmentItemID'] == 'A010001001']

In [ ]:
assess_time = train_df.groupby('assessmentItemID').prior_elapsed.mean()
assess_time.name = 'mean_elapsed'

In [ ]:
assess_time

In [ ]:
a = train_df.groupby('testId').prior_elapsed.mean()

In [ ]:
a.name = 'test_time'

In [ ]:
a

In [ ]:
train_df = train_df.merge(a, how='left', on='testId')

In [ ]:
train_df['test_mean'] = train_df.groupby('testId').prior_elapsed.transform(lambda x:x.mean())

In [ ]:
train_df[['answerCode', 'test_mean']].corr()

In [ ]:
train_df['test_mean'].plot(kind='kde')

In [ ]:
train_df['log_test_mean'] = np.log1p(train_df['test_mean'])

In [ ]:
train_df['log_test_mean'].plot(kind='kde')

In [ ]:
train_df[['answerCode', 'log_test_mean']].corr()

In [ ]:
data = train_df.groupby(['userID', 'answerCode', 'Timestamp']).apply(
    lambda x: (
        x['answerCode'].values,
        x['Timestamp'].values,
    )
)

In [ ]:
random.seed(42)
random.shuffle(data)

In [ ]:
size = int(len(data) * 0.7)
data_1 = data[:size]
data_2 = data[size:]

In [ ]:
# train / val 의 마지막 정답비율을 일치시킨다

In [ ]:

total = np.array([])
for user in data_1:
    answer = user[1][-1]
    total = np.append(total, answer)
    

In [ ]:
total.mean()

In [ ]:
total = np.array([])
for user in data_2:
    answer = user[1][-1]
    total = np.append(total, answer)


In [ ]:
total.mean()

In [ ]:
train_df.groupby('userID').answerCode.last().mean()

In [ ]:
(x for x in range(10))

In [ ]:
last_assess = train_df.groupby(['userID']).last()['assessmentItemID'].reset_index()

In [ ]:
train_df.loc[ (train_df['userID'] == 7436) & (train_df['assessmentItemID'] == 'A030019001') ]

In [ ]:
all_right = 0
better_right = 0
worse_fail = 0
all_fail = 0

for i, val in last_assess.iterrows():
    bb = train_df.loc[ (train_df['userID'] == val.userID) & (train_df['assessmentItemID'] == val.assessmentItemID) ]
    if len(bb) > 1:
        answerCode = bb.iloc[-1]['answerCode']
        prior_answerCode = bb.iloc[-2]['answerCode']
        
        if prior_answerCode == 0:
            if answerCode == 1:
                better_right += 1
            else:
                all_fail += 1
        else:
            if answerCode == 1:
                all_right += 1
            else:
                worse_fail += 1


In [ ]:
all_right

In [ ]:
better_right

In [ ]:
worse_fail

In [ ]:
all_fail

In [ ]:
36 / (45+36+36+30)

In [ ]:
last_assess

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df['grade'] = train_df.testId.str[2]

In [ ]:
train_df['grade'] = train_df['grade'].astype(int)

In [ ]:
train_df.groupby('userID').grade.mean()

In [ ]:
len(train_df.groupby('userID').grade.mean())

In [ ]:
st = np.random.randint(10, size=(6698,2))

In [ ]:
st

In [ ]:
st

In [ ]:
t, v = train_test_split(train_df.groupby('userID').grade.mean(), train_size=0.7, stratify=st)

In [ ]:
t

In [ ]:
v

In [ ]:
len(t) / (len(t) + len(v))

In [ ]:
train_df.iloc[v].grade.mean()

In [ ]:
train_df.iloc[t].grade.mean()

In [ ]:
train_df.groupby('userID').grade.mean()

In [ ]:
test_df.head(1)

In [ ]:
user_group = test_df.groupby('userID')

test_df['grade'] = test_df.testId.str[2]

test_df['grade'] = test_df['grade'].astype(int)

In [ ]:
user_group.last().grade.plot(kind='kde')

In [ ]:
user_group.answerCode.mean().plot(kind='kde')
plt.axvline(user_group.answerCode.mean().median())

In [ ]:
user_group.answerCode.mean().mean()

In [ ]:
user_group.answerCode.mean().median()

In [ ]:
user_group = train_df.groupby('userID')

train_df['grade'] = train_df.testId.str[2]

train_df['grade'] = train_df['grade'].astype(int)

In [ ]:
user_group.answerCode.mean().plot(kind='kde')
plt.axvline(user_group.answerCode.mean().median())

In [ ]:
user_group.answerCode.last().plot(kind='kde')
# plt.axvline(user_group.answerCode.last().median())

In [ ]:
r = user_group.answerCode.last()

In [ ]:
r.describe()

In [ ]:
user_group.count().grade.plot(kind='kde')

In [ ]:
test_group = test_df.groupby('userID')

In [ ]:
test_group.count().grade.plot(kind='kde')

In [ ]:
### big mean을 맞춰라

In [ ]:
train_df['big'] = train_df.testId.str[2].astype(int)
test_df['big'] = test_df.testId.str[2].astype(int)

In [ ]:
train_df.groupby('big').answerCode.mean()

In [ ]:
test_df.groupby('big').answerCode.mean()

In [ ]:
plt.plot(train_df.groupby('big').answerCode.mean())
plt.plot(test_df.groupby('big').answerCode.mean())

In [ ]:
train_df.big.value_counts()

In [ ]:
train_df.big.value_counts() / len(train_df)

In [ ]:
test_df.big.value_counts()

In [ ]:
test_df.big.value_counts() / len(test_df)

In [ ]:
test_df.groupby(['big']).answerCode.mean()

In [ ]:
train_df['split'] = (train_df['big']*10 + train_df['answerCode']).astype(int)

In [ ]:
len(train_df) * 0.2 * 0.047152

In [ ]:
21375 / 119385

In [ ]:
len(train_df) * 0.2 * 0.047152 * 0.498492

In [ ]:
0.2 * 0.047152 * 0.498492

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df.split.value_counts()

In [ ]:
train_df[train_df.split == 90]

In [ ]:
t, v = train_test_split(train_df[train_df.split == 90], test_size=0.17)

In [ ]:
len(t)

In [ ]:
len(v)

In [ ]:
### 구하기

In [ ]:
train_len = len(train_df)

In [ ]:
test_count_ratio = (test_df.big.value_counts() / len(test_df)).sort_index()

In [ ]:
test_count_ratio

In [ ]:
test_correct_ratio = test_df.groupby('big').answerCode.mean()

In [ ]:
test_correct_ratio

In [ ]:
ratio = 0.5

In [ ]:
train_ratio = [0]
valid_ratio = [0]
for grade in range(1, 10):
    count_needed = train_len * ratio * test_count_ratio[grade] * test_correct_ratio[grade]
    train_ratio.append(count_needed / train_count[grade])
    
    count_needed = train_len * ratio * test_count_ratio[grade] * (1 - test_correct_ratio[grade])
    valid_ratio.append(count_needed / train_count[grade])
    

In [ ]:
train_ratio

In [ ]:
valid_ratio

In [ ]:
grouped = train_df.groupby('userID').last()

In [ ]:
len(grouped)

In [ ]:
train_set = []
valid_set = []
for grade in range(1, 10):
    valid, train = train_test_split( train_df[train_df['split'] == (grade * 10)].index.values.tolist() , train_size=train_ratio[grade])
    train_set += train
    valid_set += valid
    
    valid, train = train_test_split( train_df[train_df['split'] == (grade * 10) + 1].index.values.tolist() , train_size=valid_ratio[grade])
    train_set += train
    valid_set += valid
    
    

In [ ]:
train_df.iloc[train_set].shape

In [ ]:
train_df.iloc[valid_set].shape

In [ ]:
len(valid_set)

In [ ]:
len(train_df)

In [ ]:
len(valid_set) / train_len

In [ ]:
len(train_set) / train_len

In [ ]:
###

In [ ]:
last_index = test_df.groupby('userID').last().index

In [ ]:
drop_last = test_df.drop(index=last_index)

In [ ]:
drop_last.groupby('userID').answerCode.mean().mean()

In [ ]:
train_df.groupby('userID').answerCode.mean().mean()

In [ ]:
t, v = train_test_split(train_df.userID.unique(), train_size=0.7, random_state=0)

In [ ]:
len(t)

In [ ]:
len(v)

In [ ]:
train_df.loc[train_df['userID'].isin(t)].groupby('userID').answerCode.mean().mean()

In [ ]:
train_df.loc[~train_df['userID'].isin(v)].groupby('userID').answerCode.mean().mean()

In [ ]:
while True:
    t, v = train_test_split(train_df.userID.unique(), train_size=0.7, random_state=0)
    m = train_df.loc[~train_df['userID'].isin(v)].groupby('userID').answerCode.mean().mean()
    if m > 0.605 and m < 0.615:
        break

In [ ]:
train_df['answer_mean'] = train_df.groupby('userID').answerCode.transform(lambda x: x.mean())

In [ ]:
lasts = train_df.groupby('userID').last().reset_index()

In [ ]:
prob = 0.
user_ids = []

In [ ]:
upper_index = lasts[lasts.answer_mean > 0.61].index.tolist()
under_index = lasts[lasts.answer_mean <= 0.61].index.tolist()

In [ ]:
len(under_index)

In [ ]:
len(upper_index)

In [ ]:
for _ in range(int(len(lasts) * 0.2)):
    if prob > 0.61:
        i = under_index.pop(0)
    else:
        i = upper_index.pop(0)
    
    user_ids.append(i)
    prob = lasts.iloc[user_ids].answer_mean.mean()
    
    

In [ ]:
len(user_ids)

In [ ]:
len(under_index)

In [ ]:
len(upper_index)

In [ ]:
prob

In [ ]:
a = train_df.groupby('userID').last().reset_index()

In [ ]:
a

In [ ]:
lasts[lasts['userID'].isin(user_ids)].answerCode

In [ ]:
train_df.groupby('userID').cumcount()

In [ ]:
cum['cummean'] = cum['answer_mean']['cumsum'] / cum['answer_mean']['cumcount']

In [ ]:
cum

In [ ]:
train_df['tmp_index'] = train_df.index
last_index = train_df.groupby('userID').last()
train_df = train_df.drop(index=last_index['tmp_index'])

In [ ]:
last_index = train_df.groupby('userID').last()

In [ ]:
last_index.reset_index()